In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import re 
from sklearn.preprocessing import OneHotEncoder

from PIL import Image
import cv2

import tensorflow as tf
from tensorflow.keras.layers import MaxPooling2D
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, Dense, Dropout, Flatten, Activation
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

In [2]:
train_filenames = []
test_filenames = []

for filename in glob.iglob('train/*/*'):
    train_filenames.append(filename)
    
for filename in glob.iglob('test/*/*'):
    test_filenames.append(filename)

In [3]:
# preprocessing steps: 
# normalize values
# convert to grayscale
# create augmented data in an imagedatagenerator
# 
# transfer learning perhaps
# CNN with last layer being softmax

In [4]:
train_images = []
train_labels = []

test_images = []
test_labels = []

for filename in train_filenames:
    image = cv2.imread(filename)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = gray / 255

    train_images.append(gray)
    
    label = re.search('(?<=\\\)([a-z]+)(?=\\\)', filename).group(0)
    train_labels.append(label)
    
for filename in test_filenames:
    image = cv2.imread(filename)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = gray / 255
    
    test_images.append(gray)
    
    label = re.search('(?<=\\\)([a-z]+)(?=\\\)', filename).group(0)
    test_labels.append(label)
    
train_images = np.array(train_images)
test_images = np.array(test_images)

In [5]:
num_classes = 7

onehot_encoder = OneHotEncoder(sparse=False)

train_labels = onehot_encoder.fit_transform(np.array(train_labels).reshape(-1, 1))
test_labels = onehot_encoder.fit_transform(np.array(test_labels).reshape(-1, 1))

### Randomly shuffle the train and test images and simultaneously shuffle the respective labels in unison

In [6]:
p1 = np.random.permutation(len(train_images))
train_images = train_images[p1]
train_labels = train_labels[p1]

p2 = np.random.permutation(len(test_images))
test_images = test_images[p2]
test_labels = test_labels[p2]

In [7]:
#cv2.imshow('img', train_images[0])

#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [53]:
model = Sequential()

model.add(Conv2D(64, kernel_size=5, activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_180 (Conv2D)         (None, 44, 44, 64)        1664      
                                                                 
 conv2d_181 (Conv2D)         (None, 40, 40, 64)        102464    
                                                                 
 max_pooling2d_92 (MaxPoolin  (None, 20, 20, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_182 (Conv2D)         (None, 18, 18, 32)        18464     
                                                                 
 conv2d_183 (Conv2D)         (None, 16, 16, 32)        9248      
                                                                 
 max_pooling2d_93 (MaxPoolin  (None, 8, 8, 32)         0         
 g2D)                                                

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist = model.fit(train_images, train_labels, validation_split=0.2, batch_size=128, epochs=100)

Epoch 1/100
180/180 [==============================] - 109s 602ms/step - loss: 1.8787 - accuracy: 0.2078 - val_loss: 1.8084 - val_accuracy: 0.2450
Epoch 2/100
180/180 [==============================] - 107s 592ms/step - loss: 1.7567 - accuracy: 0.2836 - val_loss: 1.6728 - val_accuracy: 0.3304
Epoch 3/100
180/180 [==============================] - 100s 555ms/step - loss: 1.6443 - accuracy: 0.3452 - val_loss: 1.5957 - val_accuracy: 0.3610
Epoch 4/100
180/180 [==============================] - 108s 598ms/step - loss: 1.5938 - accuracy: 0.3689 - val_loss: 1.5544 - val_accuracy: 0.3656
Epoch 5/100
180/180 [==============================] - 112s 624ms/step - loss: 1.5430 - accuracy: 0.3849 - val_loss: 1.5102 - val_accuracy: 0.3826
Epoch 6/100
180/180 [==============================] - 110s 612ms/step - loss: 1.5049 - accuracy: 0.3929 - val_loss: 1.4859 - val_accuracy: 0.3981
Epoch 7/100
180/180 [==============================] - 112s 620ms/step - loss: 1.4690 - accuracy: 0.4072 - val_loss: 1

116/180 [==================>...........] - ETA: 35s - loss: 0.7678 - accuracy: 0.7010

In [ ]:
scores = model.evaluate(test_images, test_labels, batch_size=128)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.plot(hist.history['accuracy'], ls='dashed')
ax.set_xlabel("epochs")
ax.plot(hist.history['val_' + 'accuracy'])

In [ ]:
### Attempting to improve accuracy below via two tries
# Try 1 is augmenting dataset
# Try 2 is using transfer learning

In [ ]:
datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

it_train = datagen.flow(train_images, train_labels, batch_size=64, subset='training')
it_val = datagen.flow(train_images, train_labels, batch_size=16, subset='validation')

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=[48, 48, 1])
vgg_model.trainable=False

model = Sequential([
    vgg_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu')
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
hist = model.fit(it_train, validation_data=it_val, epochs=100)